Kenan Stredic 
CS 4371 HW3

Question 1

In [0]:
# Load the text file
text_rdd = sc.textFile("dbfs:/FileStore/shared_uploads/kstredic02@hotmail.com/input_hw1.txt")

# Data preprocessing: Split lines into words, convert to lowercase, and remove punctuation
import re
word_rdd = text_rdd.flatMap(lambda line: re.split(r'\W+', line.lower()))

# Word Count
word_count = word_rdd.map(lambda word: (word, 1)).reduceByKey(lambda a, b: a + b)

# Specific Word Counts
specific_words = ["america", "president", "washington"]
specific_word_counts = word_count.filter(lambda x: x[0] in specific_words)

# Top 10 Words
top_10_words = word_count.takeOrdered(10, key=lambda x: -x[1])

# Save Question 1 results to one output text file
combined_question1_results = word_count.union(specific_word_counts).union(sc.parallelize(top_10_words))
combined_question1_results.coalesce(1).saveAsTextFile("/FileStore/shared_uploads/kstredic02@hotmail.com/question1_combined_results.txt")

Out[48]: 512

Question 2

In [0]:
# Load the input files
temperature_rdd = sc.textFile("dbfs:/FileStore/shared_uploads/kstredic02@hotmail.com/city_temperature.csv")
country_list_rdd = sc.textFile("dbfs:/FileStore/shared_uploads/kstredic02@hotmail.com/country_list.csv")

# Data preprocessing for temperature data
temperature_data = temperature_rdd.map(lambda line: line.split(","))
temperature_data = temperature_data.filter(lambda row: row[0] != "Region")  # Remove header

# Data preprocessing for country list data
country_data = country_list_rdd.map(lambda line: line.split(","))
country_data = country_data.filter(lambda row: row[2] == "country")  # Filter by type

# Sub-Question A - Average Temperature by Region
region_avg_temp = temperature_data.map(lambda row: (row[0], (float(row[7]), 1)))
region_avg_temp = region_avg_temp.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
region_avg_temp = region_avg_temp.map(lambda x: (x[0], x[1][0] / x[1][1]))

# Sub-Question B - Average Temperature by Year (Asia Region)
asia_avg_temp = temperature_data.filter(lambda row: row[0] == "Asia")
asia_avg_temp = asia_avg_temp.map(lambda row: ((row[6], row[7]), float(row[7])))
asia_avg_temp = asia_avg_temp.reduceByKey(lambda a, b: a + b)
asia_avg_temp = asia_avg_temp.map(lambda x: (x[0][0], (x[0][1], x[1])))
asia_avg_temp = asia_avg_temp.reduceByKey(lambda a, b: (a[0], a[1] + b[1]))
asia_avg_temp = asia_avg_temp.map(lambda x: (x[0], x[1][1] / len(x[1][0])))

# Sub-Question C - Average Temperature by City (Spain)
spain_avg_temp = temperature_data.filter(lambda row: row[2] == "Spain")
spain_avg_temp = spain_avg_temp.map(lambda row: (row[3], float(row[7])))
spain_avg_temp = spain_avg_temp.reduceByKey(lambda a, b: a + b)
spain_avg_temp = spain_avg_temp.map(lambda x: (x[0], x[1] / len(x[1])))
                                    
# Sub-Question D - Capital and Average Temperature
capital_avg_temp = country_data.join(temperature_data.map(lambda row: (row[1], (float(row[7]), 1))))
capital_avg_temp = capital_avg_temp.map(lambda x: (x[1][0], x[1][1]))
capital_avg_temp = capital_avg_temp.reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))
capital_avg_temp = capital_avg_temp.filter(lambda x: x[1][1] > 0)
capital_avg_temp = capital_avg_temp.map(lambda x: (x[0], x[1][0] / x[1][1]))

# Save Question 2 results to another output text file
combined_question2_results = region_avg_temp.union(asia_avg_temp).union(spain_avg_temp).union(capital_avg_temp)
combined_question2_results.coalesce(1).saveAsTextFile("/FileStore/shared_uploads/kstredic02@hotmail.com/question2_combined_results.txt")